In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pwd

In [ ]:
in_mov =pd.read_csv("Invmovements.csv",sep = ",")
# data = pd.read_csv("Invmovements.csv",sep = ",")

In [ ]:
in_mov.head()

In [ ]:
in_mov = in_mov[in_mov['Plant']=='C004']

In [ ]:
in_mov.StorageLocation.unique()

In [ ]:
in_mov.StorageLocation.value_counts()

In [ ]:
#in_mov = in_mov[in_mov['StorageLocation'].isin([30,35])]

In [ ]:
in_mov = in_mov[in_mov['MovementType'].isin([261])]

In [ ]:
in_mov.head()

In [ ]:
in_mov.PostingDate = in_mov.PostingDate.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))

In [ ]:
 in_mov['mnth_yr'] = in_mov['PostingDate'].apply(lambda x: x.strftime('%B-%Y')) 

In [ ]:
in_mov.mnth_yr.head()

In [ ]:
monthly_agg_data = pd.DataFrame({'Qty_consumed':in_mov.groupby(['MaterialNumber_MARC','mnth_yr'])['Quantity_SUM'].sum(),
                                'mov_count':in_mov.groupby(['MaterialNumber_MARC','mnth_yr'])['PostingDate'].count()})

In [ ]:
monthly_agg_data.reset_index(inplace=True)

In [ ]:
monthly_agg_data.head()

In [ ]:
mavg_qty_freq = pd.DataFrame({'mavg_qty':monthly_agg_data.groupby(['MaterialNumber_MARC'])['Qty_consumed'].mean(),
                              'Freq':monthly_agg_data.groupby(['MaterialNumber_MARC'])['mov_count'].mean()})

In [ ]:
mavg_qty_freq.head()

In [ ]:
mavg_qty_freq.reset_index(inplace=True)

In [ ]:
import seaborn as sns

In [ ]:
mavg_qty_freq['mavg_qty'].describe()

In [ ]:
mavg_qty_freq['mavg_qty'].quantile([.75,.80,.85,.90,.95,.96,.97,.98,.99])

In [ ]:
#cutoffs for low, medium,high volume are 0-10,10-50,50-max 

In [ ]:
low_volume = mavg_qty_freq[mavg_qty_freq['mavg_qty']<10]['MaterialNumber_MARC'].tolist()

In [ ]:
medium_volume = mavg_qty_freq[(mavg_qty_freq['mavg_qty']>=10) & (mavg_qty_freq['mavg_qty']<=50)]['MaterialNumber_MARC'].tolist()

In [ ]:
high_volume = mavg_qty_freq[mavg_qty_freq['mavg_qty']>50]['MaterialNumber_MARC'].tolist()

In [ ]:
q = mavg_qty_freq[mavg_qty_freq['mavg_qty']<60]
plt.figure(figsize=(16,8))
sns.distplot(q['mavg_qty'],kde=False)
plt.axvline(x=10,alpha=1,color='black')
plt.axvline(x=50,alpha=1,color='black')
plt.ylabel('Count')
plt.title("Volume_cutoffs")
plt.savefig("analysis_plots/" + "Volume_cutoffs.png", format="PNG")  

In [ ]:
mavg_qty_freq['Freq'].describe()

In [ ]:
mavg_qty_freq['Freq'].quantile([.75,.80,.85,.90,.95,.96,.97,.98,.99])

In [ ]:
#cutoffs for low, medium,high volume are 0-1.67,1.67-5,5-max 

In [ ]:
low_frequency = mavg_qty_freq[mavg_qty_freq['Freq']<1.67]['MaterialNumber_MARC'].tolist()

In [ ]:
medium_frequency = mavg_qty_freq[(mavg_qty_freq['Freq']>=1.67) & (mavg_qty_freq['Freq']<=5)]['MaterialNumber_MARC'].tolist()

In [ ]:
high_frequency = mavg_qty_freq[mavg_qty_freq['Freq']>5]['MaterialNumber_MARC'].tolist()

In [ ]:
f = mavg_qty_freq[mavg_qty_freq['Freq']<10]
plt.figure(figsize=(16,8))
sns.distplot(f['Freq'],kde=False)

plt.axvline(x=1.67,alpha=1,color='black')
plt.axvline(x=5,alpha=1,color='black')

plt.ylabel('Count')
plt.title("frequency_cutoffs")
plt.savefig("analysis_plots/" + "frequency_cutoffs.png", format="PNG")  

# Time Series

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import datetime
import numpy as np
import pandas as pd
import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm

In [ ]:
data = pd.read_csv("Invmovements.csv",sep = ",")

In [ ]:
data.head()

In [ ]:
df_mov = data[data["Plant"] == "C004"]
df_mov.shape

In [ ]:
df_mov = df_mov[df_mov.MovementType.isin([261])]

In [ ]:
df_mov.rename(columns = {"MaterialNumber_MARC": "MaterialNumber"}, 
                                 inplace = True) 
df_mov  = df_mov[["PostingDate","StorageLocation","MaterialNumber","MovementType","DebitCreditInd","Quantity_SUM"]]

df_mov.shape

In [ ]:
grp_mov = df_mov.groupby(["MaterialNumber","MovementType","PostingDate"]).sum()

In [ ]:
grp_mov = grp_mov.reset_index()

In [ ]:
grp_mov.PostingDate = grp_mov.PostingDate.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))

In [ ]:
grp_mov = grp_mov.sort_values(by = 'PostingDate')
del grp_mov["StorageLocation"]
grp_mov.head()

In [ ]:
del grp_mov["MovementType"]
grp_mov.head()

In [ ]:
grp_mov.shape

In [ ]:
##n_series to find the individual series for a kunag,matnr combination
def n_series(input_df,matnr): 
    df = input_df.copy()
#     print(df.head())
#     print("matnr :",matnr)
    n_df1 = df[(df["MaterialNumber"] == matnr)]
#     print(n_df1.head())

    #sort date start to end
    n_df1 = n_df1.sort_values('PostingDate')
    n_df1.set_index('PostingDate',inplace=True)
#     print(n_df1.head())

    #sampling the data on weekly basis (index is set to date first in the above step to do weekly sampling
    weekly_resampled_data = n_df1.Quantity_SUM.resample('M').sum() 
    weekly_resampled_data = weekly_resampled_data.replace(np.nan, 0)
    individual_series = weekly_resampled_data.to_frame()     
    #resetting index so that date can be used as column
    individual_series = individual_series.reset_index()
    return individual_series

#splitting the individual series basesd on the number of weeks (i, is for dyanamic point forecasts)
def train_test_split(input_df,matnr,i):
    data= n_series(input_df,matnr)
    train = data.iloc[0:-i]    
    test = data.iloc[-i:]
    return train,test

In [ ]:
def moving_average(input_df,matnr,n,roll):

    index = str(matnr)
    i = 0
    lst = []
    test1 = train_test_split(input_df,matnr,n)[1]
#     y_hat = test1.copy()
    y_hat_avg = test1.copy()

    for i in range(n,0,-1):
        train,test = train_test_split(input_df,matnr,i)
        dd= np.asarray(train["Quantity_SUM"])
        y_hat_avg['moving_avg_forecast'] = train['Quantity_SUM'].rolling(roll).mean().iloc[-1]
        pred = y_hat_avg['moving_avg_forecast']
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst
    plt.figure(figsize=(12,8))
    plt.plot( train.set_index("PostingDate")['Quantity_SUM'], label='Train',marker = '.')
    plt.plot(test1.set_index("PostingDate")['Quantity_SUM'], label='Test',marker = '.')
    plt.plot(y_hat_avg.set_index("PostingDate")['pred_column'], label='moving_avg_forecast',marker = '.')
    plt.legend(loc='best')
    plt.title(str(matnr)+"_moving_avg_forecast")
    plt.savefig( "plots_ma12/" + 'Graph_{}.png'.format(index), format="PNG")  

#     plt.show()
    rms = sqrt(mean_squared_error(test1.Quantity_SUM, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.Quantity_SUM, y_hat_avg.pred_column)
    del y_hat_avg['moving_avg_forecast']
    return y_hat_avg,rms,mae


In [ ]:
def arima(input_df,matnr,n,p,d,q):

    index =str(matnr)
    i = 0
    lst = []
    test1 = train_test_split(input_df,matnr,n)[1]
    y_hat_avg = test1.copy()
    start_date = str(test1["PostingDate"][:1])
    end_date = str(test1["PostingDate"][-1:])
    order = (p, d,q)
    for i in range(n,0,-1):
        train,test = train_test_split(input_df,matnr,i)
        dd= np.asarray(train["Quantity_SUM"])
        fit1 = sm.tsa.statespace.SARIMAX(train["Quantity_SUM"], order=order,enforce_stationarity = False, enforce_invertibility = False,trend = "n").fit()    
        pred = fit1.predict(1)
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst
    plt.figure(figsize=(12,8))
    plt.plot( train.set_index("PostingDate")['Quantity_SUM'], label='Train',marker = '.')
    plt.plot(test1.set_index("PostingDate")['Quantity_SUM'], label='Test',marker = '.')
    plt.plot(y_hat_avg.set_index("PostingDate")['pred_column'], label='ARIMA' + "_" + str(order),marker = '.')
    plt.legend(loc='best')
    plt.title(str(matnr)+"_ARIMA")

    plt.savefig("plots_arima12/" + 'Graph_{}.png'.format(index), format="PNG")  

#     plt.show()
    rms = sqrt(mean_squared_error(test1.Quantity_SUM, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.Quantity_SUM, y_hat_avg.pred_column)
    return y_hat_avg,rms,mae


In [ ]:
#Simple Exponential Smoothing
def ses(input_df,matnr,n,alpha):

    index =  str(matnr)
    i = 0
    lst = []
    test1 = train_test_split(input_df,matnr,n)[1]
    y_hat_avg = test1.copy()
    for i in range(n,0,-1):
        train,test = train_test_split(input_df,matnr,i)
        dd= np.asarray(train["Quantity_SUM"])
        fit2 = SimpleExpSmoothing(np.asarray(train['Quantity_SUM'])).fit(smoothing_level=alpha,optimized=False)
        y_hat_avg['SES'] = fit2.forecast(len(test1))
        pred = y_hat_avg['SES']
        lst.append(pred.iloc[-1])

    pd.DataFrame(lst)
    y_hat_avg['pred_column']=lst
    plt.figure(figsize=(12,8))
    plt.plot( train.set_index("PostingDate")['Quantity_SUM'], label='Train')
    plt.plot(test1.set_index("PostingDate")['Quantity_SUM'], label='Test')
    plt.plot(y_hat_avg.set_index("PostingDate")['pred_column'], label='SES',marker = ".")
    plt.legend(loc='best')
    plt.title(str(matnr)+"_SES")
    plt.savefig("plots_ses12/" + 'Graph_{}.png'.format(index), format="PNG")  

#     plt.show()
    rms = sqrt(mean_squared_error(test1.Quantity_SUM, y_hat_avg.pred_column))
    mae = mean_absolute_error(test1.Quantity_SUM, y_hat_avg.pred_column)
    del y_hat_avg['SES']
    return y_hat_avg,rms,mae


In [ ]:
len(low_frequency)

In [ ]:
len(medium_frequency)

In [ ]:
len(high_frequency)

In [ ]:
# moving_average(grp_mov,150000005,4,4)

# Low Frequency Data

In [ ]:
for i in range (0,50):
            try:
                matnr = low_frequency[i]
                print("index : ",matnr)
                output,rms,mae = moving_average(grp_mov,matnr,12,4)
                print("moving_average :",rms)
            except :
                pass

# Medium Frequency Data

In [ ]:
for i in range (0,50):
            try:
                matnr = medium_frequency[i]
                print("index : ",matnr)
                output3,rms3,mae3 = ses(grp_mov,matnr,12,0.2)
                print("arimas :",rms3)
            except :
                pass

# High Frequency Data

In [ ]:
for i in range (0,59):
            try:
                matnr = high_frequency[i]
                print("index : ",matnr)
                output,rms,mae = arima(grp_mov,matnr,12,0,1,1)
                print("arima :",rms)
            except :
                pass

In [ ]:
columns = "011","101","110","111","121","112","211"
df_res= pd.DataFrame(index=None, columns=columns)

In [ ]:
df_res

In [ ]:
def_red =pd.DataFrame(OrderedDict({"matnr" : [matnr],
                    "011" : [round(rms1,3)],
                   "101" : [round(rms2,3)],
                   "110" : [round(rms3,3)],
                   "111" : [round(rms4,3)],
                   "211" : [round(rms5,3)],
                   "121" : [round(rms6,3)],
                   "112" : [round(rms7,3)]}))
            print(df_res)
            main_df = main_df.append(df_res, ignore_index = True) 

## Eperimentations


In [ ]:
columns = "011","101","110","111","121","112","211"
df_res= pd.DataFrame(index=None, columns=columns)
print(df_res)
for i in range (0,20):
            try:
                matnr = high_frequency[i]
                print("index : ",matnr)
                output1,rms1,mae1 = ses(grp_mov,matnr,4,0.2)
                output2,rms2,mae2 = ses(grp_mov,matnr,4,0.3)
                output3,rms3,mae3 = ses(grp_mov,matnr,4,0.4)
                output4,rms4,mae4 = ses(grp_mov,matnr,4,0.5)
                output5,rms5,mae5 = ses(grp_mov,matnr,4,0.6)
                output6,rms6,mae6 = ses(grp_mov,matnr,4,0.1)
                output7,rms7,mae7 = ses(grp_mov,matnr,4,0.05)
                df_res =pd.DataFrame(({"matnr" : [matnr],
                    "011" : [round(rms1,3)],
                    "101" : [round(rms2,3)],
                   "110" : [round(rms3,3)],
                   "111" : [round(rms4,3)],
                   "211" : [round(rms5,3)],
                   "121" : [round(rms6,3)],
                   "112" : [round(rms7,3)]}))
                print(df_res)
                main_df = df_res.append(result_df, ignore_index = True) 
                print(main_df)

#                 print("arima :",rms)
            except :
                pass

In [ ]:
main_df